In [2]:
!git clone https://github.com/matterport/Mask_RCNN

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pwd
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/M6/week3")
!ls
!pwd

/content
“aidl2019_cv_lab2_segmentation.ipynb”的副本  Mask_RCNN		utils_w3.py
Datasets				     __pycache__	work.ipynb
Datasets.zip				     utils_Gaussian.py
/content/drive/My Drive/Colab Notebooks/M6/week3


In [5]:
import os
import numpy as np
import sys
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import utils_w3 as utw3
from utils_Gaussian import read_video_and_divide, calculate_mean_std_first_part_video, calculate_mask, find_detections
from tqdm import tqdm
# Root directory of the project
ROOT_DIR = os.path.abspath("./Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log


#%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# # Path to Shapes trained weights
SHAPES_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_shapes.h5")
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))
print (ROOT_DIR)
import coco
config = coco.CocoConfig()
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
TEST_MODE = "inference"


def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.

    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size * cols, size * rows))
    return ax
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config = config)

# Set weights file path
if config.NAME == "shapes":
    weights_path = SHAPES_MODEL_PATH
elif config.NAME == "coco":
    weights_path = COCO_MODEL_PATH
# Or, uncomment to load the last model you trained
# weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)


/content/drive/My Drive/Colab Notebooks/M6/week3/Mask_RCNN

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss':

Using TensorFlow backend.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights  /content/drive/My Drive/Colab Notebooks/M6/week3/Mask_RCNN/mask_rcnn_coco.h5








In [0]:
'''
Test the network with different images taken from our dataset. Take into account that the network can only predict classes from COCO.
'''
video_length = 2141
# video_length = 2141
# video_length = 100
video_split_ratio = 0.25
# video_path = "./Datasets/AICity_data/train/S03/c010/vdo.avi"
video_path = "./Datasets/AICity/frames/"
groundtruth_xml_path = 'Datasets/AICity/aicity_annotations.xml'
# groundtruth_path = "../datasets/AICity_data/train/S03/c010/gt/gt.txt"
roi_path = 'Datasets/AICity_data/train/S03/c010/roi.jpg'
# roi = cv2.cvtColor(cv2.imread(roi_path))
ver = True
# print("Reading annotations...")
# groundTruth = utw3.read_annotations(groundtruth_xml_path, video_length)
# gt_filtered = [x for x in groundTruth if x['frame'] > int(video_length * video_split_ratio)]
# video_first_part, video_second_part, divide_frame = \
#     read_video_and_divide(video_path, video_length=video_length, video_split_ratio=video_split_ratio)



In [8]:
detections = []
frame_id = 0
for i in tqdm(range(video_length)):
    frame_id = frame_id + 1

    filename = "{}{}.jpg".format(video_path, str(i + 1).zfill(5))

    image = cv2.imread(filename)

    # Run detection
    results = model.detect([image], verbose=0)
    r = results[0]
    class_id = r['class_ids']
    # print (class_id)
    boxes = r['rois']
    confidence = r['scores']
    for j in range(class_id.shape[0]):
        if class_id[j] == 3:
            bbox = boxes[j, :]
            detection = {}
            box_h = bbox[2] - bbox[0]
            box_w = bbox[3] - bbox[1]
            detection['frame'] = frame_id
            detection['left'] = bbox[1]
            detection['top'] = bbox[0]
            detection['width'] = box_w
            detection['height'] = box_h
            detection['confidence'] = confidence[j]
            detections.append(detection)


100%|██████████| 2141/2141 [39:01<00:00,  1.15s/it]


In [10]:
for detection in detections:
  print (detection)

流式输出内容被截断，只能显示最后 5000 行内容。
{'frame': 1414, 'left': 561, 'top': 89, 'width': 101, 'height': 79, 'confidence': 0.9077573}
{'frame': 1414, 'left': 896, 'top': 82, 'width': 116, 'height': 62, 'confidence': 0.8576292}
{'frame': 1414, 'left': 441, 'top': 135, 'width': 31, 'height': 39, 'confidence': 0.75358075}
{'frame': 1415, 'left': 1297, 'top': 360, 'width': 225, 'height': 177, 'confidence': 0.9947337}
{'frame': 1415, 'left': 1176, 'top': 103, 'width': 52, 'height': 57, 'confidence': 0.9812665}
{'frame': 1415, 'left': 561, 'top': 93, 'width': 99, 'height': 74, 'confidence': 0.93000954}
{'frame': 1415, 'left': 897, 'top': 81, 'width': 116, 'height': 65, 'confidence': 0.8617858}
{'frame': 1415, 'left': 442, 'top': 139, 'width': 30, 'height': 34, 'confidence': 0.7034069}
{'frame': 1416, 'left': 1297, 'top': 360, 'width': 225, 'height': 177, 'confidence': 0.99463356}
{'frame': 1416, 'left': 1176, 'top': 103, 'width': 53, 'height': 57, 'confidence': 0.9817121}
{'frame': 1416, 'left': 561, 'top

In [0]:
import pickle
with open("detections.pkl", 'wb') as f:
    pickle.dump(detections, f)
    f.close()